## ADD DESCRIPTION FOR THE PROJECT ##

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras_tuner as kt
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras import regularizers
from keras import metrics

from sklearn.utils import shuffle
from sklearn import model_selection

2022-11-14 17:58:07.911051: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-14 17:58:08.245626: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-14 17:58:08.245647: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-14 17:58:08.298417: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-14 17:58:09.440972: W tensorflow/stream_executor/platform/de

Loading data for the ML cup 2022

In [57]:
colnames = ['id', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'class1', 'class2']
mlcup_tr = pd.read_csv("./dataset/ml_cup22/ML-CUP22-TR.csv", sep = ",", names=colnames)
mlcup_tr = mlcup_tr.iloc[7:, :]
mlcup_tr = mlcup_tr.drop('id', axis=1)
mlcup_tr = (mlcup_tr-mlcup_tr.min())/(mlcup_tr.max()-mlcup_tr.min())

In [58]:
x_mlcup_tr = mlcup_tr.iloc[:, 0:9].values
y_mlcup_tr = mlcup_tr.iloc[:, 9:11].values

## Neural Network with Random Search ##

Below we extended the RandomSearch class of the keras_tuner library to perform a cross validation during the phase of model selection (which is not supported by the RandomSearch class).

In [59]:
# ELIMINAREEEEEEEEEEEE
X = np.random.random((500, 9))
y = np.random.random((500, 2))
X_val = np.random.random((100, 9))
y_val = np.random.random((100, 2))

In [60]:
class CrossValidationRandomSearch(kt.RandomSearch):

    
    cross_validation_partitions = -1
    
    
    def __init__(
        self,
        hypermodel=None,
        objective=None,
        max_trials=10,
        seed=None,
        hyperparameters=None,
        tune_new_entries=True,
        allow_new_entries=True,
        folds_number=1,
        **kwargs
    ):
        self.cross_validation_partitions = model_selection.KFold(folds_number)
        
        
        kt.RandomSearch.__init__(
            self,
            hypermodel=hypermodel,
            objective=objective,
            max_trials=max_trials,
            seed=seed,
            hyperparameters=hyperparameters,
            tune_new_entries=tune_new_entries,
            allow_new_entries=allow_new_entries,
            **kwargs
        )

        
    def run_trial(self, trial, x_train, y_train, *args, **kwargs):
    
        histories_cv = []
        histories_folds = []
        # build a different model for each fold of the cross validation
        for train_indices, test_indices in self.cross_validation_partitions.split(x_train):
            x_cv_train, x_cv_val = x_train[train_indices], x_train[test_indices]
            y_cv_train, y_cv_val = y_train[train_indices], y_train[test_indices]
            histories_folds = super(CrossValidationRandomSearch, self).run_trial(
                trial, x_cv_train, y_cv_train, validation_data=(x_cv_val, y_cv_val), *args, **kwargs
            )
            histories_cv.append(histories_folds)

            histories_folds = []
        # return the performance of each model
        return histories_cv

Below we defined two functions; the first one is used to create a model given a combination of hyperparameters, while the second one select a random combination of hyperparameters in a given range

In [61]:
# instantiate a model given a specific combination of hyperparameters
def set_hyperparameters(units, ridge_lambda, learning_rate, momentum, double_layer):
    model = Sequential()
    model.add(
        Dense(
            units=units,
            activity_regularizer=regularizers.L2(ridge_lambda),
            activation="relu",
        )
    )
    # if double_layer == True, a second layer for the NN is added
    if (double_layer):
        model.add(
            Dense(
                units=units,
                activity_regularizer=regularizers.L2(ridge_lambda),
                activation="relu"
            )
        )
    model.add(Dense(2, activation="linear", activity_regularizer=regularizers.L2(ridge_lambda)))
    # the optimizer is the stochastic gradient descent algorithm
    opt = optimizers.SGD(learning_rate=learning_rate, momentum=momentum)
    model.compile(
        optimizer=opt, loss="mean_squared_error", metrics=["mean_absolute_error"]
    )
    
    return model

# extract a random combination of hyperparamaters
def build_regressor(hp, min_units=2, max_units=11, min_ridge_lambda=0.0001, max_ridge_lambda=0.1, min_lr_rate=0.1, 
                max_lr_rate=0.6, min_mom=0.0, max_mom=0.9, linear_reg=False, double_layer=False):
    
    ridge_lambda_sampling = "log"
    if(linear_reg):
        ridge_lambda_sampling = "linear"
    
    units_step = 1
    if (max_units != min_units):
        units_step = 3
    
    # the variable 'units' represents the number of units of the hidden layer
    # by default this value is extracted between 2 and 11 (the user can specify a different interval)
    units = hp.Int("units", min_value=min_units, max_value=max_units, step=units_step)
    
    # the variable 'ridge_lambda' represents the lambda coefficient for ridge regularization
    # by default this value is extracted between 0.0001 and 0.1 with a logarithmic probabilistic law
    # (the user can specify a different interval)
    ridge_lambda = hp.Float(
        "ridge_lambda", min_value=min_ridge_lambda, max_value=max_ridge_lambda, sampling=ridge_lambda_sampling
    )
    
    # the variable 'learning rate' represents the step used by the stochastic gradient descent algorithm
    # by default this value is extracted between 0.1 and 0.6 (the user can specify a different interval)
    learning_rate = hp.Float("learning_rate", min_value=min_lr_rate, max_value=max_lr_rate, sampling="linear")
    
    # the variable 'momentum' represents the coefficient drawn for momentum
    # by default this value is extracted between 0.1 and 0.6 (the user can specify a different interval)
    momentum = hp.Float("momentum", min_value=min_mom, max_value=max_mom, sampling="linear")
    
    # Create the model with the set of hyperparameters drew
    model = set_hyperparameters(
        units=units, ridge_lambda=ridge_lambda, learning_rate=learning_rate, 
        momentum=momentum, double_layer=double_layer
    )
    return model

Let's start searching randomly the best combination of hyperparameters using only one hidden layer.

In [62]:
tuner = CrossValidationRandomSearch(
    hypermodel=build_regressor, # function that return a model given a combination of hyperparameters
    objective=kt.Objective("val_mean_absolute_error", direction="min"), # value to minimize
    max_trials=50, # maximum number of attempts
    executions_per_trial=2, # number of models created for each fold of the cross validation
    overwrite=True,
    directory="results/ml_cup",
    project_name="nn_RandomSearch/par_hp",
    folds_number=5 # number of folds of the cross validation
)

print a brief description of the search space

In [63]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 11, 'step': 3, 'sampling': None}
ridge_lambda (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.1, 'step': None, 'sampling': 'log'}
learning_rate (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.6, 'step': None, 'sampling': 'linear'}
momentum (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.9, 'step': None, 'sampling': 'linear'}


In [ ]:
tuner.search(
    # the training set for the model selection
    x_mlcup_tr, y_mlcup_tr,# maximum number of epochs allowed
    epochs=1000,
    # callback to implement early stopping
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_mean_absolute_error',  patience=10)]
)

Trial 6 Complete [00h 00m 45s]
val_mean_absolute_error: 0.08996025919914245

Best val_mean_absolute_error So Far: 0.06927253715693951
Total elapsed time: 00h 04m 05s

Search: Running Trial #7

Value             |Best Value So Far |Hyperparameter
8                 |8                 |units
0.0010079         |0.0013868         |ridge_lambda
0.44811           |0.23573           |learning_rate
0.6174            |0.88412           |momentum

Epoch 1/1000
38/38 [==============================] - 1s 8ms/step - loss: 0.1220 - mean_absolute_error: 0.2550 - val_loss: 0.0667 - val_mean_absolute_error: 0.2128
Epoch 2/1000
38/38 [==============================] - 0s 3ms/step - loss: 0.0680 - mean_absolute_error: 0.2158 - val_loss: 0.0653 - val_mean_absolute_error: 0.2088
Epoch 3/1000
38/38 [==============================] - 0s 3ms/step - loss: 0.0680 - mean_absolute_error: 0.2166 - val_loss: 0.0734 - val_mean_absolute_error: 0.2215
Epoch 4/1000
38/38 [==============================] - 0s 3ms/step -

38/38 [==============================] - 0s 2ms/step - loss: 0.0090 - mean_absolute_error: 0.0687 - val_loss: 0.0123 - val_mean_absolute_error: 0.0853
Epoch 23/1000
38/38 [==============================] - 0s 3ms/step - loss: 0.0093 - mean_absolute_error: 0.0697 - val_loss: 0.0093 - val_mean_absolute_error: 0.0690
Epoch 24/1000
38/38 [==============================] - 0s 2ms/step - loss: 0.0090 - mean_absolute_error: 0.0690 - val_loss: 0.0093 - val_mean_absolute_error: 0.0693
Epoch 25/1000
38/38 [==============================] - 0s 2ms/step - loss: 0.0094 - mean_absolute_error: 0.0701 - val_loss: 0.0105 - val_mean_absolute_error: 0.0763
Epoch 26/1000
38/38 [==============================] - 0s 2ms/step - loss: 0.0092 - mean_absolute_error: 0.0704 - val_loss: 0.0094 - val_mean_absolute_error: 0.0697
Epoch 27/1000
38/38 [==============================] - 0s 2ms/step - loss: 0.0088 - mean_absolute_error: 0.0683 - val_loss: 0.0107 - val_mean_absolute_error: 0.0754
Epoch 28/1000
38/38 [===

In [54]:
len(y_mlcup_tr[0])

2